In [1]:
# Analytical Tools
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# General Utilities
import math
import json
import pprint
import itertools
import importlib

# Some settings
from IPython.display import Markdown, display
log = pprint.pprint
space = lambda: print()
printmd = lambda text: display(Markdown(text))
%matplotlib inline

def set_pandas_display_options() -> None:
    """Set pandas display options."""
    display = pd.options.display

    display.max_columns = 1000
    display.max_rows = 1000
    display.max_colwidth = 199
    display.width = 1000
    # display.precision = 2  # set as needed

set_pandas_display_options()

In [63]:
non_insight = ['Three spies, suspected as double agents, speak as follows when questioned: Albert: "Bertie is a mole." Bertie: "Cedric is a mole." Cedric: "Bertie is lying." Assuming that moles lie, other agents tell the truth, and there is just one mole among the three, who is the mole? Albert, Bertie, or Cedric?',
	'What day follows the day before yesterday if two days from now will be Sunday?',
	'A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now facing?',
	'There are three playing cards lying face up, side by side. A five is just to the right of a two. A five is just to the left of a two. A spade is just to the left of a club, and a spade is just to the right of a spade. What are the three cards?',
	'The following verse spells out a word. I am a food for you and me. My first letter is in pat but not in tar. My second letter is in stream but not in smart. My third letter is in giraffe but not in fridge. My fourth letter is in treat but not in absent. What is the word that this verse described?',
	'Next week I am going to have lunch with my friend, visit the new art gallery, go to the Social Security office, and have my teeth checked at the dentist. My friend cannot meet me on Wednesday; the Social Security office is closed weekends; the dentist has office hours only on Tuesday, Friday, and Saturday; the art gallery is closed Tuesday, Thursday, and weekends. On what single day can I do everything I have planned?',
	'Ben spent $42 for shoes. This was $14 less than what he spent for a shirt and twice more expensive than the tie. The tie was $20 cheaper than the jeans. How much was the jeans?',
	'Three people play a game in which one person loses and two people win each round. The one who loses must double the amount of money that each of the other two players has at that time. The three players agree to play three games. At the end of the three games, each player has lost one game and each person has $8. What was the original stake of each player?',
	 'Andrew is half the age of Brian, Brian is three times older than Charles and the sum of their ages is 44 years. How old is Charles?'];

insight = ['A man and his son are in a serious car accident. The father is killed and the son is rushed to the emergency room. Upon arrival, the attending doctor looks at the child and screams: \'This child is my son!\' Who is the doctor?',
'Joe Fan has no psychic powers but he can tell you the score of any football game before it starts. How?',
'A man bought a horse for $60 and sold it for $70. Then he bought it back for $80 and sold it for $90. How much did he make or lose in the horse trading business?',
'A man is reading a book when the lights go off but even although the room is pitch dark the man goes on reading. How?',
'A woman gave natural birth to two sons who were born on the same hour of the same day of the same month of the same year. But they were not twins and she had no access to a time machine. How could this be?',
'Mr. Hardy slipped and fell off a sixty-foot ladder onto the concrete floor below. However, he did not injure himself in any way. How is this possible?',
'A murderer is condemned to death. He has to choose among three rooms. The first is full of raging fires, the second is full of assassins with loaded guns, and the third is full of lions that haven\'t eaten in 3 years. Which room is safest for him?',
'Alex and Casey are blood relatives of Bobbie. However, Alex and Casey are not blood relatives at all. How is this possible?',
'Sid Shady was working for a large construction company that was very concerned about employee theft. Someone tipped company security that Shady was the man to watch. Each night, he passed through security with a wheelbarrow full of scrap lumber, discarded electrical wires, and chunks of concrete. The security guards checked the contents daily, but could find nothing of value. What was Shady stealing?'];
 
anagram = ['Solve this anagram: LUBMEJD', 
						'Solve this anagram: AALRYS',
						'Solve this anagram: NAYUGHT',
						'Solve this anagram: MARNOD',
						'Solve this anagram: HAKMOCM',
						'Solve this anagram: EVANG',
						'Solve this anagram: SAOSI',
						'Solve this anagram: AYKAWLJ',
						'Solve this anagram: NCRBOA']

solutions1 = 'Bertie|Thursday|South|Pear|41|4|2|Friday|five spade|5 spade|2 spade|5 club|8'

solutions2 = 'jumbled|Salary|Naughty|Random|Hammock|Oasis|Jaywalk|Carbon|Vegan'

In [64]:
with open('pilot2.json') as json_file:
    data = json.load(json_file)    

In [65]:
questions_non_insight = []
answers_non_insight = []
confidence_non_insight = []
difficulty_non_insight = []
aha_non_insight = []
rts_non_insight = []

questions_insight = []
answers_insight = []
confidence_insight = []
difficulty_insight = []
aha_insight = []
rts_insight = []

questions_anagram = []
answers_anagram = []
confidence_anagram = []
difficulty_anagram = []
aha_anagram = []
rts_anagram = []

for i in range(204): #go thru the subjects
    if len(data[i]['data']) > 3: #only look at subjects that finished the task (and not just started without finishing)
        if int(data[i]['data'][3]['responses'][7]) == 6: #passed attention/quiz check, hooray!
            
            for j in range(6): #go through the 6 problems shown to people
                question = data[i]['data'][0]['problems_seen'][j] #store the question seen by the participant
                answer = data[i]['data'][j*3+5]['responses'] #store people's answers to the questions here
                rt = data[i]['data'][j*3+5]['rt']/1000 #store people's RT for the answer provided
                response = data[i]['data'][j*3+6]['responses'] #store people's responses to the questions here
                
                if question in non_insight:   
                    questions_non_insight.append(question)
                    answers_non_insight.append(answer)
                    rts_non_insight.append(rt)
                    confidence_non_insight.append(int(response[6])+1)
                    difficulty_non_insight.append(int(response[13])+1)
                    aha_non_insight.append(int(response[20])+1)
                    
                elif question in insight:
                    questions_insight.append(question)
                    answers_insight.append(answer)
                    rts_insight.append(rt)
                    confidence_insight.append(int(response[6])+1)
                    difficulty_insight.append(int(response[13])+1)
                    aha_insight.append(int(response[20])+1)
                
                elif question in anagram:
                    questions_anagram.append(question)
                    answers_anagram.append(answer)
                    rts_anagram.append(rt)
                    confidence_anagram.append(int(response[6])+1)
                    difficulty_anagram.append(int(response[13])+1)
                    aha_anagram.append(int(response[20])+1)
        else:     
            print('bad subject!')

bad subject!


Convert data into df

In [66]:
d = {'questions':questions_non_insight,
     'answers':answers_non_insight, 
     'rts': rts_non_insight,
     'confidence': confidence_non_insight, 
     'difficulty': difficulty_non_insight, 
     'aha': aha_non_insight}

d2 = {'questions':questions_insight,
     'answers':answers_insight, 
      'rts': rts_insight,
     'confidence': confidence_insight, 
     'difficulty': difficulty_insight, 
     'aha': aha_insight}

d3 = {'questions':questions_anagram,
     'answers':answers_anagram, 
     'rts': rts_anagram,
     'confidence': confidence_anagram, 
     'difficulty': difficulty_anagram, 
     'aha': aha_anagram}

df_non_insight = pd.DataFrame(d)
df_insight = pd.DataFrame(d2)
df_anagram = pd.DataFrame(d3)

df_non_insight_filtered = df_non_insight.loc[(df_non_insight['answers'].str.contains(solutions1, case = False)==True)]
df_anagram_filtered = df_anagram.loc[(df_anagram['answers'].str.contains(solutions2, case = False)==True)]


In [67]:
df1 = df_non_insight.groupby(['questions'], as_index=False).agg(['mean', 'count'])
df2 = df_non_insight_filtered.groupby(['questions']).agg(['mean', 'count'])
df3 = df_anagram.groupby(['questions']).agg(['mean', 'count'])
df4 = df_anagram_filtered.groupby(['questions']).agg(['mean', 'count'])


In [95]:
print(df2.index.values)
print('% Solved')
print(df2['rts']['count'].values/df1['rts']['count'].values)
print('Mean RT')
print(df2['rts']['mean'].values)
print('Mean difficulty')
print(df2['difficulty']['mean'].values)

['A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now facing?'
 'Andrew is half the age of Brian, Brian is three times older than Charles and the sum of their ages is 44 years. How old is Charles?'
 'Ben spent $42 for shoes. This was $14 less than what he spent for a shirt and twice more expensive than the tie. The tie was $20 cheaper than the jeans. How much was the jeans?'
 'Next week I am going to have lunch with my friend, visit the new art gallery, go to the Social Security office, and have my teeth checked at the dentist. My friend cannot meet me on Wednesday; the Social Security office is closed weekends; the dentist has office hours only on Tuesday, Friday, and Saturday; the art gallery is closed Tuesday, Thursday, and weekends. On what single day can I do everything I have planned?'
 'The following verse spells out a word. I am a food for

In [82]:
print('% Solved')
print(df4['rts']['count']/df3['rts']['count'])
print('Mean RT')
print(df4['rts']['mean'])
print('Mean difficulty')
print(df4['difficulty']['mean'])

% Solved
questions
Solve this anagram: AALRYS     0.739130
Solve this anagram: AYKAWLJ    0.703704
Solve this anagram: EVANG      0.666667
Solve this anagram: HAKMOCM    0.880000
Solve this anagram: LUBMEJD    0.857143
Solve this anagram: MARNOD     0.400000
Solve this anagram: NAYUGHT    0.904762
Solve this anagram: NCRBOA     0.642857
Solve this anagram: SAOSI      0.842105
Name: count, dtype: float64
Mean RT
questions
Solve this anagram: AALRYS     45.164529
Solve this anagram: AYKAWLJ    55.031368
Solve this anagram: EVANG      59.167000
Solve this anagram: HAKMOCM    37.170818
Solve this anagram: LUBMEJD    34.777167
Solve this anagram: MARNOD     56.021000
Solve this anagram: NAYUGHT    15.968842
Solve this anagram: NCRBOA     60.222222
Solve this anagram: SAOSI      36.742500
Name: mean, dtype: float64
Mean difficulty
questions
Solve this anagram: AALRYS     2.529412
Solve this anagram: AYKAWLJ    2.526316
Solve this anagram: EVANG      2.875000
Solve this anagram: HAKMOCM    2.

In [94]:
df2.index.values

array(['A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now facing?',
       'Andrew is half the age of Brian, Brian is three times older than Charles and the sum of their ages is 44 years. How old is Charles?',
       'Ben spent $42 for shoes. This was $14 less than what he spent for a shirt and twice more expensive than the tie. The tie was $20 cheaper than the jeans. How much was the jeans?',
       'Next week I am going to have lunch with my friend, visit the new art gallery, go to the Social Security office, and have my teeth checked at the dentist. My friend cannot meet me on Wednesday; the Social Security office is closed weekends; the dentist has office hours only on Tuesday, Friday, and Saturday; the art gallery is closed Tuesday, Thursday, and weekends. On what single day can I do everything I have planned?',
       'The following verse 

Analyze only those responses for which people's confidence is greater than 5

In [45]:
print(df_non_insight["aha"].mean(),df_insight["aha"].mean(), df_anagram["aha"].mean())
print(df_non_insight["difficulty"].mean(),df_insight["difficulty"].mean(), df_anagram["difficulty"].mean())
print(df_non_insight["confidence"].mean(),df_insight["confidence"].mean(), df_anagram["confidence"].mean())

2.423469387755102 2.566326530612245 3.443877551020408
4.035714285714286 3.816326530612245 3.5816326530612246
4.739795918367347 4.285714285714286 5.204081632653061


In [81]:
print(df_non_insight_filtered["aha"].mean(), df_anagram_filtered["aha"].mean())
print(df_non_insight_filtered["difficulty"].mean(), df_anagram_filtered["difficulty"].mean())
print(df_non_insight_filtered["confidence"].mean(), df_anagram_filtered["confidence"].mean())

2.595419847328244 4.1931034482758625
3.564885496183206 2.703448275862069
5.312977099236641 6.517241379310345


Plot distribution of curiosity ratings below

In [9]:
df_non_insight


,questions,answers,rts,confidence,difficulty,aha
0,A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now fa...,"{""Q0"":""east""}",22.237,2,5,1
1,"There are three playing cards lying face up, side by side. A five is just to the right of a two. A five is just to the left of a two. A spade is just to the left of a club, and a spade is just to...","{""Q0"":""""}",12.844,1,5,1
2,"Next week I am going to have lunch with my friend, visit the new art gallery, go to the Social Security office, and have my teeth checked at the dentist. My friend cannot meet me on Wednesday; th...","{""Q0"":""Friday""}",31.739,6,6,4
3,What day follows the day before yesterday if two days from now will be Sunday?,"{""Q0"":""Wednesday""}",32.094,6,2,2
4,The following verse spells out a word. I am a food for you and me. My first letter is in pat but not in tar. My second letter is in stream but not in smart. My third letter is in giraffe but not ...,"{""Q0"":""pear""}",39.578,6,2,1
5,Ben spent $42 for shoes. This was $14 less than what he spent for a shirt and twice more expensive than the tie. The tie was $20 cheaper than the jeans. How much was the jeans?,"{""Q0"":""41""}",101.659,6,3,1
6,"There are three playing cards lying face up, side by side. A five is just to the right of a two. A five is just to the left of a two. A spade is just to the left of a club, and a spade is just to...","{""Q0"":""5 of spades. 2 of spades, 5 of clubs""}",45.818,6,5,3
7,A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now fa...,"{""Q0"":""West""}",19.790,5,4,1
8,A group of soldiers were standing facing west. Their sergeant shouted at them: Right turn! U-turn! Left turn! U-turn! Right turn! U-turn! Left turn! Right turn! In which direction are they now fa...,"{""Q0"":""NORTH ""}",34.074,4,4,1
9,Ben spent $42 for shoes. This was $14 less than what he spent for a shirt and twice more expensive than the tie. The tie was $20 cheaper than the jeans. How much was the jeans?,"{""Q0"":""next""}",14.911,1,7,1


In [ ]:
b/2 + b + b/3 = 44 --> b = 4*6 = 24/3 = 8